# Train MOTOR

This tutorial walks through the various steps to train a MOTOR model.

Training MOTOR is a four step process:

- Training a tokenizer
- Prefitting MOTOR
- Preparing batches
- Training the model

In [1]:
import shutil
import os

TARGET_DIR = 'trash/tutorial_6'

if os.path.exists(TARGET_DIR):
    shutil.rmtree(TARGET_DIR)

os.mkdir(TARGET_DIR)

In [2]:
import meds_reader
import femr.splits

# First, we want to split our dataset into train, valid, and test
# We do this by calling our split functionality twice

database = meds_reader.SubjectDatabase('input/meds_reader')

main_split = femr.splits.generate_hash_split(list(database), 97, frac_test=0.15)

os.mkdir(os.path.join(TARGET_DIR, 'motor_model'))
# Note that we want to save this to the target directory since this is important information

main_split.save_to_csv(os.path.join(TARGET_DIR, "motor_model", "main_split.csv"))

train_split = femr.splits.generate_hash_split(main_split.train_subject_ids, 87, frac_test=0.15)

main_database = database.filter(main_split.train_subject_ids)
train_database = main_database.filter(train_split.train_subject_ids)
val_database = main_database.filter(train_split.test_subject_ids)


In [3]:
import femr.models.tokenizer
import pickle

# First, we need to train a tokenizer
# Note, we need to use a hierarchical tokenizer for MOTOR

with open('input/ontology.pkl', 'rb') as f:
    ontology = pickle.load(f)

# NOTE: A vocab size of 128 is probably too low for a real model. 128 was chosen to make this tutorial quick to run
tokenizer = femr.models.tokenizer.train_tokenizer(
    train_database, vocab_size=128, is_hierarchical=True, ontology=ontology)

# Save the tokenizer to the same directory as the model
tokenizer.save_pretrained(os.path.join(TARGET_DIR, "motor_model"))

/home/ethanid/health_research/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

import femr.models.tasks

# Second, we need to prefit the MOTOR model. This is necessary because piecewise exponential models are unstable without an initial fit

motor_task = femr.models.tasks.MOTORTask.fit_pretraining_task_info(
    train_database, tokenizer, num_tasks=64, num_bins=4, final_layer_size=32)

# It's recommended to save this with pickle to avoid recomputing since it's an expensive operation


In [7]:
import femr.models.processor
import femr.models.tasks

# Third, we need to create batches. 

processor = femr.models.processor.FEMRBatchProcessor(tokenizer, motor_task)

example_subject_id = list(train_database)[0]
example_subject = train_database[example_subject_id]

# We can do this one subject at a time
print("Convert a single subject")
example_batch = processor.collate([processor.convert_subject(example_subject, tensor_type='pt')])

print("Convert batches")
# But generally we want to convert entire datasets
train_batches = processor.convert_dataset(train_database, tokens_per_batch=32, num_proc=4)

print("Convert batches to pytorch")
# Convert our batches to pytorch tensors
train_batches.set_format("pt")
print("Done")

val_batches = processor.convert_dataset(val_database, tokens_per_batch=32, num_proc=4)
# Convert our batches to pytorch tensors
val_batches.set_format("pt")

Convert a single subject
Convert batches
Creating batches 8


Generating train split: 8 examples [00:00, 26.46 examples/s]


Convert batches to pytorch
Done
Creating batches 1


Setting num_proc from 4 back to 1 for the train split to disable multiprocessing as it only contains one shard.
Generating train split: 1 examples [00:00, 172.15 examples/s]


In [8]:
import transformers

import femr.models.transformer

# Finally, given the batches, we can train CLMBR.
# We can use huggingface's trainer to do this.

transformer_config = femr.models.config.FEMRTransformerConfig(
    vocab_size=tokenizer.vocab_size, 
    is_hierarchical=tokenizer.is_hierarchical, 
    n_layers=2,
    hidden_size=64, 
    intermediate_size=64*2,
    n_heads=8,
)

config = femr.models.config.FEMRModelConfig.from_transformer_task_configs(transformer_config, motor_task.get_task_config())

model = femr.models.transformer.FEMRModel(config)

collator = processor.collate

trainer_config = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,

    output_dir='tmp_trainer',
    remove_unused_columns=False,
    num_train_epochs=100,

    eval_steps=20,
    evaluation_strategy="steps",

    logging_steps=20,
    logging_strategy='steps',

    prediction_loss_only=True,
)

trainer = transformers.Trainer(
    model=model,
    data_collator=processor.collate,
    train_dataset=train_batches,
    eval_dataset=val_batches,
    args=trainer_config,
)

trainer.train()

model.save_pretrained(os.path.join(TARGET_DIR, 'motor_model'))

/home/ethanid/health_research/venv/lib/python3.12/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss
20,0.830100,0.523818
40,0.810400,0.523785
60,0.809800,0.523750
80,0.829200,0.523720
100,0.827000,0.523700
120,0.810600,0.523686
140,0.798000,0.523673
160,0.838300,0.523665
180,0.827100,0.523665
200,0.808000,0.523664
